In [ ]:
import pymysql
from datetime import datetime

def date_to_timestamp_ms(date_str):
    """날짜 문자열을 유닉스 타임스탬프 밀리초로 변환"""
    dt = datetime.strptime(date_str, '%Y-%m-%d')
    return int(dt.timestamp() * 1000)

def get_source_ids_by_date_range(start_date, end_date):
    """
    특정 기간의 source_id를 조회하여 cmds 파일 생성
    
    Args:
        start_date: 시작일 (예: '2020-01-01')
        end_date: 종료일 (예: '2020-10-30')
    """
    # 날짜를 타임스탬프로 변환
    start_ts = date_to_timestamp_ms(start_date)
    # 종료일은 23:59:59.999까지 포함
    end_dt = datetime.strptime(end_date, '%Y-%m-%d')
    end_ts = int(datetime(end_dt.year, end_dt.month, end_dt.day, 23, 59, 59, 999999).timestamp() * 1000)
    
    print(f"조회 기간: {start_date} ~ {end_date}")
    print(f"타임스탬프 범위: {start_ts} ~ {end_ts}")
    
    # MariaDB 연결
    connection = pymysql.connect(
        host='localhost',      # DB 호스트 주소
        port=3306,             # 포트
        user='root',  # 사용자명
        password='1234',  # 비밀번호
        database='any_approval',
        charset='utf8mb4'
    )
    
    try:
        with connection.cursor() as cursor:
            # source_id 조회
            sql = """
                SELECT source_id 
                FROM documents 
                WHERE created_at >= %s AND created_at <= %s
                ORDER BY source_id
            """
            cursor.execute(sql, (start_ts, end_ts))
            results = cursor.fetchall()
            
            print(f"조회된 레코드 수: {len(results)}")
            
            # cmds 파일 생성
            output_filename = f'delete_commands_{start_date}_to_{end_date}.cmds'
            with open(output_filename, 'w', encoding='utf-8') as f:
                for row in results:
                    source_id = row[0]
                    f.write(f'deleteDocument {{"sourceId":"{source_id}"}}\n')
            
            print(f"파일 생성 완료: {output_filename}")
            
    finally:
        connection.close()

# 사용 예시
if __name__ == '__main__':
    # 날짜 입력
    start_date = input("시작일 (YYYY-MM-DD): ").strip()
    end_date = input("종료일 (YYYY-MM-DD): ").strip()
    
    # 또는 직접 지정
    # start_date = '2020-01-01'
    # end_date = '2020-10-30'
    
    get_source_ids_by_date_range(start_date, end_date)